In [1]:
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import camb
from camb import model, initialpower
import math
from scipy.integrate import simps as simps

In [2]:
pars = camb.CAMBparams()
pars.set_cosmology(H0 = 72)
results = camb.get_background(pars)

In [ ]:
def alpha_distance(z):
    some code here    
    return D_alpha

In [13]:
# check this implementation agrees with results from standard CAMB lum distance function
Hubble_param = 72
omega_m = .3
omega_lambda = .7

def func( z_prime ):
    return 1 / np.sqrt( z_prime ) # issue because math sqrt fctn converts array into scalar, use np.sqrt

def func2( z_prime ):
    return (omega_m * (1 + a * z_prime)**3 + omega_lambda)

def integrand( z_prime ):
    return func(func2( z_prime ))

a = np.array(range(0,50)) # controls number of points integrand is evaluated at
z_prime = np.linspace(0,4).reshape(-1,1)
integral = simps(integrand(z_prime), z_prime, axis=0)


def lum_dist( z ):
    constant_term = (1 + z) / Hubble_param
    return constant_term * integral

lum_dist(4)

array([0.27777778, 0.11626657, 0.07250538, 0.0530627 , 0.04200167,
       0.0348332 , 0.02979704, 0.02605898, 0.02317157, 0.02087279,
       0.01899879, 0.01744171, 0.01612762, 0.01500403, 0.01403267,
       0.01318494, 0.01243901, 0.01177791, 0.01118828, 0.0106594 ,
       0.01018262, 0.00975082, 0.00935815, 0.0089997 , 0.00867135,
       0.00836962, 0.00809153, 0.00783452, 0.00759639, 0.00737522,
       0.00716936, 0.00697735, 0.00679789, 0.00662986, 0.00647226,
       0.00632419, 0.00618487, 0.00605357, 0.00592966, 0.00581257,
       0.00570179, 0.00559684, 0.00549729, 0.00540278, 0.00531294,
       0.00522745, 0.00514604, 0.00506842, 0.00499435, 0.00492362])

In [12]:
# calculate luminosity distances for redshifts 0 to 4
z = np.linspace(0,4)
camb_Dl = results.luminosity_distance(z)
camb_Dl

array([    0.        ,   361.33350322,   763.20244116,  1202.18060569,
        1674.98880621,  2178.54672677,  2710.00518535,  3266.76141467,
        3846.4609177 ,  4446.98959139,  5066.45944702,  5703.19064644,
        6355.69190817,  7022.64072766,  7702.86435405,  8395.32208093,
        9099.08913283,  9813.34224122, 10537.34688413, 11270.44609213,
       12012.05068687, 12761.63080278, 13518.70854242, 14282.85162268,
       15053.66788097, 15830.80052419, 16613.92401673, 17402.74051739,
       18196.9767872 , 18996.38150086, 19800.72290461, 20609.78677124,
       21423.37461058, 22241.3020998 , 23063.39770311, 23889.50145516,
       24719.46388591, 25553.14506827, 26390.41377245, 27231.14671309,
       28075.22787757, 28922.54792517, 29773.00364844, 30626.49748925,
       31482.93710297, 32342.23496512, 33204.30801567, 34069.0773367 ,
       34936.46785961, 35806.40809881])

In [10]:
len(lum_dist(4))


50

In [11]:
len(camb_Dl)

50